# 📋 Estructura del Proyecto - 5 Etapas

Este notebook está organizado en 5 etapas claramente definidas para facilitar su comprensión y ejecución:

## 🔧 Etapa 1: Configuración y Descarga de Datos
- Instalación de dependencias necesarias (Transformers, PyTorch, etc.)
- Verificación de GPU y configuración del dispositivo
- Descarga automática de textos del Proyecto Gutenberg

## 📊 Etapa 2: Preprocesamiento y Análisis Exploratorio
- Limpieza y tokenización de textos
- Análisis del corpus y vocabulario
- Preparación de secuencias para el modelo LSTM

## 🏗️ Etapa 3: Arquitectura del Modelo
- Fine-tuning de GPT-2 para generación de texto
- Configuración de tokenizador y modelo preentrenado
- Preparación para entrenamiento con Transformers

## 🚀 Etapa 4: Entrenamiento
- Entrenamiento del modelo por 5 épocas
- Monitoreo en tiempo real del progreso
- Guardado del mejor modelo

## 📈 Etapa 5: Evaluación y Resultados
- Generación de texto automático
- Análisis de la calidad del texto generado
- Visualización de métricas de rendimiento

---

# Generación de Texto Automático - Project Gutenberg
## Framework: Transformers (Sin TensorFlow)
### Generar texto similar al estilo de autores clásicos usando GPT-2

In [ ]:
# ⚠️ ADVERTENCIA: Configuración de Dependencias para Generación de Texto con Transformers
# Este notebook usa Transformers (Hugging Face) + PyTorch (NO TensorFlow)
# Optimizado para Python 3.8-3.11. En Python 3.13 pueden existir incompatibilidades.

import subprocess
import sys
import warnings
warnings.filterwarnings('ignore')

print("🔧 Instalando dependencias para generación de texto con Transformers...")
print(f"📋 Python version: {sys.version}")

# Instalar numpy primero con versión compatible
packages_to_install = [
    "numpy>=1.21.0,<2.0.0",  # Versión específica para compatibilidad
    "torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118",
    "transformers>=4.20.0",
    "datasets",
    "accelerate",
    "nltk",
    "matplotlib",
    "seaborn",
    "scikit-learn",
    "tqdm"
]

for package in packages_to_install:
    print(f"📦 Instalando: {package}")
    try:
        result = subprocess.run([sys.executable, "-m", "pip", "install"] + package.split(),
                              capture_output=True, text=True, check=True)
        print(f"✅ {package.split()[0]} instalado correctamente")
    except subprocess.CalledProcessError as e:
        print(f"❌ Error instalando {package}: {e}")
        print(f"Output: {e.stdout}")
        print(f"Error: {e.stderr}")

print("\n🔍 Verificando instalación de PyTorch y CUDA...")
try:
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")
    print(f"🚀 CUDA disponible: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"🎮 GPU detectada: {torch.cuda.get_device_name(0)}")
        print(f"💾 Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    else:
        print("⚠️ CUDA no disponible - usando CPU")
except ImportError as e:
    print(f"❌ Error importando PyTorch: {e}")

print("\n✅ Proceso de instalación completado!")

🔧 Instalando dependencias para generación de texto con Transformers...
📋 Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
📦 Instalando: numpy>=1.21.0,<2.0.0
✅ numpy>=1.21.0,<2.0.0 instalado correctamente
📦 Instalando: torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
✅ torch instalado correctamente
📦 Instalando: transformers>=4.20.0
✅ transformers>=4.20.0 instalado correctamente
📦 Instalando: datasets
✅ datasets instalado correctamente
📦 Instalando: accelerate
✅ accelerate instalado correctamente
📦 Instalando: nltk
✅ nltk instalado correctamente
📦 Instalando: matplotlib
✅ matplotlib instalado correctamente
📦 Instalando: seaborn
✅ seaborn instalado correctamente
📦 Instalando: scikit-learn
✅ scikit-learn instalado correctamente
📦 Instalando: tqdm
✅ tqdm instalado correctamente

🔍 Verificando instalación de PyTorch y CUDA...
✅ PyTorch version: 2.6.0+cu118
🚀 CUDA disponible: False
⚠️ CUDA no disponible - usando CPU

✅ Proceso de instalación c

In [ ]:
# Importaciones para Generación de Texto con Transformers y PyTorch
import warnings
warnings.filterwarnings('ignore')
!pip install --upgrade --force-reinstall numpy matplotlib seaborn --no-cache-dir

print("📚 Importando librerías...")

# Importaciones básicas
import os
import sys
import time
import re
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Verificar y configurar PyTorch
try:
    import torch
    import torch.nn as nn
    from torch.utils.data import Dataset, DataLoader
    print(f"✅ PyTorch version: {torch.__version__}")

    # Configurar dispositivo
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"🔧 Dispositivo configurado: {device}")

    if torch.cuda.is_available():
        print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")

except ImportError as e:
    print(f"❌ Error importando PyTorch: {e}")
    sys.exit(1)

# Importar Transformers con manejo de errores
try:
    from transformers import (
        GPT2LMHeadModel,
        GPT2Tokenizer,
        TrainingArguments,
        Trainer,
        DataCollatorForLanguageModeling,
        pipeline
    )
    from datasets import Dataset as HFDataset
    print("✅ Transformers importado correctamente")

except ImportError as e:
    print(f"❌ Error importando Transformers: {e}")
    print("💡 Intenta reinstalar: pip install transformers datasets")
    # Continuar sin salir para permitir diagnóstico

# Importar NLTK
try:
    import nltk
    print("✅ NLTK importado correctamente")
except ImportError as e:
    print(f"❌ Error importando NLTK: {e}")

# Configurar reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

print("🎯 Configuración completada - Listo para generar texto!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 158.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 155.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 142.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 130.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 181.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.2/326.2 kB 175.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 161.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 191.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 139.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 176.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

📚 Importando librerías...
✅ PyTorch version: 2.6.0+cu118
🔧 Dispositivo configurado: cpu
❌ Error importando Transformers: Could not import module 'GPT2LMHeadModel'. Are this object's requirements defined correctly?
💡 Intenta reinstalar: pip install transformers datasets
✅ NLTK importado correctamente
🎯 Configuración completada - Listo para generar texto!


# 📊 ETAPA 2: PREPROCESAMIENTO Y ANÁLISIS EXPLORATORIO
---

In [ ]:
# Descargar textos de Project Gutenberg
def download_gutenberg_text(book_id, title):
    """Descargar libro de Project Gutenberg"""
    url = f"https://www.gutenberg.org/files/{book_id}/{book_id}-0.txt"

    try:
        response = requests.get(url)
        response.raise_for_status()

        # Limpiar texto
        text = response.text

        # Encontrar inicio y fin del texto principal
        start_markers = ["*** START OF", "***START OF"]
        end_markers = ["*** END OF", "***END OF"]

        start_idx = 0
        for marker in start_markers:
            idx = text.find(marker)
            if idx != -1:
                start_idx = text.find('\n', idx) + 1
                break

        end_idx = len(text)
        for marker in end_markers:
            idx = text.find(marker)
            if idx != -1:
                end_idx = idx
                break

        text = text[start_idx:end_idx]

        # Guardar archivo
        filename = f"{title.replace(' ', '_').lower()}.txt"
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(text)

        print(f"Descargado: {title} - {len(text)} caracteres")
        return filename, text

    except Exception as e:
        print(f"Error descargando {title}: {e}")
        return None, None

# Lista de libros para descargar
books = [
    (1513, "Romeo and Juliet"),
    (1524, "Hamlet"),
    (1533, "Macbeth"),
    (1540, "The Tempest"),
    (23, "A Midsummer Night's Dream")
]

# Descargar todos los libros
all_texts = []
for book_id, title in books:
    filename, text = download_gutenberg_text(book_id, title)
    if text:
        all_texts.append(text)

# Combinar todos los textos
combined_text = '\n\n'.join(all_texts)
print(f"\nTexto combinado: {len(combined_text)} caracteres")
print(f"Primeros 500 caracteres:\n{combined_text[:500]}...")

Error descargando Romeo and Juliet: name 'requests' is not defined
Error descargando Hamlet: name 'requests' is not defined
Error descargando Macbeth: name 'requests' is not defined
Error descargando The Tempest: name 'requests' is not defined
Error descargando A Midsummer Night's Dream: name 'requests' is not defined

Texto combinado: 0 caracteres
Primeros 500 caracteres:
...


In [ ]:
import re
import pickle
from transformers import GPT2Tokenizer

print("Preparando texto para fine-tuning de GPT-2...")

# Inicializar tokenizador de GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Añadir token de padding si no existe
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Tokenizador cargado: {tokenizer.name_or_path}")
print(f"Vocabulario: {len(tokenizer)} tokens")

# Limpiar y preparar texto
def clean_text_for_gpt2(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s\.\,\!\?\;\:\-\"\'\(\)]', '', text)
    text = re.sub(r'\s+([\.,:;!?])', r'\1', text)
    return text.strip()

# 👉 Asegúrate de definir esta variable:
# combined_text = "Tu texto aquí..."

clean_combined_text = clean_text_for_gpt2(combined_text)

# Tokenizar texto
print("Tokenizando texto...")
tokens = tokenizer.encode(clean_combined_text)
print(f"Texto tokenizado: {len(tokens)} tokens")

def create_text_chunks(tokens, chunk_size=512, overlap=50):
    chunks = []
    for i in range(0, len(tokens) - chunk_size, chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        if len(chunk) == chunk_size:
            chunks.append(chunk)
    return chunks

CHUNK_SIZE = 256
chunks = create_text_chunks(tokens, CHUNK_SIZE)
print(f"Creados {len(chunks)} chunks de texto para entrenamiento")

print("\n=== EJEMPLOS DE TEXTO PROCESADO ===")
for i in range(min(3, len(chunks))):
    decoded = tokenizer.decode(chunks[i][:50])
    print(f"Chunk {i+1}: {decoded}...")

preprocessed_data = {
    'chunks': chunks,
    'tokenizer_name': 'gpt2',
    'chunk_size': CHUNK_SIZE,
    'total_tokens': len(tokens)
}

with open('preprocessed_shakespeare.pkl', 'wb') as f:
    pickle.dump(preprocessed_data, f)

print("Datos preprocessados guardados exitosamente")


Preparando texto para fine-tuning de GPT-2...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Tokenizador cargado: gpt2
Vocabulario: 50257 tokens
Tokenizando texto...
Texto tokenizado: 0 tokens
Creados 0 chunks de texto para entrenamiento

=== EJEMPLOS DE TEXTO PROCESADO ===
Datos preprocessados guardados exitosamente


In [ ]:
import os
import re
import pickle
import random
import torch
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer
import urllib.request

# === Configuración ===
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Dispositivo: {device}")

# === Descargar texto si no existe ===
file_path = "shakespeare.txt"
url = "https://www.gutenberg.org/cache/epub/100/pg100.txt"

if not os.path.exists(file_path):
    print("⬇️ Descargando texto de Shakespeare...")
    urllib.request.urlretrieve(url, file_path)
    print(f"📄 Archivo guardado como {file_path}")

# === Cargar texto ===
with open(file_path, "r", encoding="utf-8") as f:
    combined_text = f.read()

print(f"📄 Texto cargado ({len(combined_text)} caracteres)")

# === Preprocesamiento ===
def clean_text_for_gpt2(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s\.\,\!\?\;\:\-\"\'\(\)]', '', text)
    text = re.sub(r'\s+([\.,:;!?])', r'\1', text)
    return text.strip()

clean_combined_text = clean_text_for_gpt2(combined_text)

# === Tokenizador ===
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokens = tokenizer.encode(clean_combined_text)
print(f"🔠 Tokens generados: {len(tokens)}")

# === Crear chunks ===
def create_text_chunks(tokens, chunk_size=128, overlap=32):
    chunks = []
    for i in range(0, len(tokens) - chunk_size, chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        if len(chunk) == chunk_size:
            chunks.append(chunk)
    return chunks

CHUNK_SIZE = 128
chunks = create_text_chunks(tokens, CHUNK_SIZE)
print(f"📦 Chunks creados: {len(chunks)}")

# === Clase Dataset personalizada ===
class ShakespeareDataset(Dataset):
    def __init__(self, chunks, tokenizer, max_length=128):
        self.chunks = chunks
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.chunks)

    def __getitem__(self, idx):
        chunk = self.chunks[idx]
        if len(chunk) > self.max_length:
            chunk = chunk[:self.max_length]
        elif len(chunk) < self.max_length:
            chunk += [self.tokenizer.pad_token_id] * (self.max_length - len(chunk))

        return {
            "input_ids": torch.tensor(chunk, dtype=torch.long),
            "attention_mask": torch.tensor([1 if token != self.tokenizer.pad_token_id else 0 for token in chunk], dtype=torch.long),
            "labels": torch.tensor(chunk, dtype=torch.long)
        }

# === Crear Dataset y dividir ===
if len(chunks) == 0:
    raise ValueError("⚠️ No se generaron chunks. Aumenta el texto o reduce el tamaño de chunk.")

dataset = ShakespeareDataset(chunks, tokenizer, CHUNK_SIZE)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
print(f"✅ Dataset creado: {len(dataset)} ejemplos")
print(f"   🔹 Train: {len(train_dataset)}")
print(f"   🔸 Val: {len(val_dataset)}")

# === Mostrar un ejemplo ===
example = dataset[0]
print("\n📌 Ejemplo de entrada:")
print(f"Input shape: {example['input_ids'].shape}")
print("Texto decodificado:")
print(tokenizer.decode(example['input_ids'][:50]))

# === Guardar datos ===
preprocessed_data = {
    "chunks": chunks,
    "tokenizer_name": "gpt2",
    "chunk_size": CHUNK_SIZE,
    "total_tokens": len(tokens)
}

with open("preprocessed_shakespeare.pkl", "wb") as f:
    pickle.dump(preprocessed_data, f)

print("\n💾 Datos preprocessados guardados en 'preprocessed_shakespeare.pkl'")


✅ Dispositivo: cpu
⬇️ Descargando texto de Shakespeare...
📄 Archivo guardado como shakespeare.txt
📄 Texto cargado (5378663 caracteres)


Token indices sequence length is longer than the specified maximum sequence length for this model (1395601 > 1024). Running this sequence through the model will result in indexing errors


🔠 Tokens generados: 1395601
📦 Chunks creados: 14537
✅ Dataset creado: 14537 ejemplos
   🔹 Train: 13083
   🔸 Val: 1454

📌 Ejemplo de entrada:
Input shape: torch.Size([128])
Texto decodificado:
The Project Gutenberg eBook of The Complete Works of William Shakespeare This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or

💾 Datos preprocessados guardados en 'preprocessed_shakespeare.pkl'
